In [53]:
import streamlit as st
import pandas as pd
from bokeh.plotting import figure,save
from bokeh.models import ColumnDataSource, CustomJS, Quad, CDSView, IndexFilter, HoverTool, Range1d, LabelSet, Label, GroupFilter, BooleanFilter
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.layouts import column, row
from lib.text_data import TextData, TextDataStore
td = TextDataStore.load(filepath="data/app_reviews_thums_up.tds.gz")

from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [15]:
import numpy as np
bins = np.linspace(1, 10, 10, )
bins

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [16]:
d = {column.column_name:column.records for column in td.aux_data.columns}
d["text"] = td.docs.texts
df = pd.DataFrame(d)
df["thums_up"] = df["thums_up"].str.split("人").apply(lambda x: x[0].replace(",", "")).astype(int).apply(lambda x:np.digitize(x, bins))
df["text_length"] = df["text"].str.len()
# df

In [17]:
from typing import Literal

xaxis_name1 = "thums_up"
xagg_axis_name1 = f"{xaxis_name1}_agg_axis"
xagg_axis1 = df[xaxis_name1].to_numpy()
xaxis_property1 = None
use_binning1 = False
# TODO: xaxis_name1 の型によって、binの設定や粒度選択や値の選択を行う
# if td.aux_data[xaxis_name1] が 数値なら、
# binの範囲や大きさを選択できるようにする。日付なら日付の粒度を選択できるようにする。
# そのほかの場合は、通常のカテゴリーと同じように扱う。

caxis_name1 = "star"
# TODO: caxis_nameは、group_nameのみを選択できるようにする。

yaxis_name1 = "name"
metric_name1:Literal["count", "nunique", "mean", "median", "sum"] = "count"

# yaxis_name1 = "text_length"
# metric_name1:Literal["count", "nunique", "mean", "median", "sum"] = "mean"
# TODO: yaxisは件数、数値型の列、そのほか任意の型の列を選択できるようにする
# 件数の場合は、countとする
# 数値型の場合は、mean, median, sum, nunique を選択できるようにする
# そのほかの場合は、nuniqueを選択できるようにする


In [18]:
import numpy as np
xaxis_name2 = "text_length"
xagg_axis_name2 = f"{xaxis_name2}_agg_axis"
use_binning2 = True
xaxis_property2 = {
    "bin_start": 0,
    "bin_end": 600,
    "nbins": 20,
    }
xbins2 = np.linspace(xaxis_property2["bin_start"], xaxis_property2["bin_end"], xaxis_property2["nbins"], )
xaxis_property2["bins"] = xbins2
xagg_axis2 = np.digitize(df[xaxis_name2], xbins2)
caxis_name2 = "star"
yaxis_name2 = "name"
metric_name2:Literal["count", "nunique", "mean", "median", "sum"] = "count"

In [19]:
# def get_df_agg(xagg_axis_name, caxis_name, yaxis_name, metric_name, df:pd.DataFrame, use_binning, xagg_axis = None):
#     df[xagg_axis_name] = xagg_axis
#     if metric_name == "count":
#         df_agg = df.groupby([xagg_axis_name, caxis_name])[yaxis_name].size().reset_index()
#     elif metric_name == "mean":
#         df_agg = df.groupby([xagg_axis_name, caxis_name])[yaxis_name].mean().reset_index()
#     elif metric_name == "median":
#         df_agg = df.groupby([xagg_axis_name, caxis_name])[yaxis_name].median().reset_index()
#     elif metric_name == "nunique":
#         df_agg = df.groupby([xagg_axis_name, caxis_name])[yaxis_name].nunique().reset_index()
#     xaxis_referer = df[xagg_axis_name].to_numpy()
#     return df_agg, xaxis_referer
#     # TODO: use_binning の場合は、 digitizeした値を xaxis_referer にする。
#     # いずれにせよ、axisについての selected indices と 等しい値のxaxis_referer のindexを取得することで、
#     # df_agg の選択された値に対応する df の行を取得できる。
# df_agg1, xaxis_referer1 = get_df_agg(xagg_axis_name1, caxis_name1, yaxis_name1, metric_name1, df, use_binning1, xagg_axis = xagg_axis1)
# df_agg2, xaxis_referer2 = get_df_agg(xagg_axis_name2, caxis_name2, yaxis_name2, metric_name2, df, use_binning2, xagg_axis = xagg_axis2)

In [36]:
# df_agg1

In [44]:
# df_agg1
# # xagg_axis_name1
# # caxis_name1
# # yaxis_name1

In [54]:
from bokeh.plotting import Figure

In [89]:
from typing import overload
@overload
def aggregate(
        df:pd.DataFrame,
        xaxis_name:str,
        yaxis_name:str,
        yaxis_aggregate_method:str,
        caxis_name:None = None,
        )->tuple[pd.Series, pd.Series]:
    ...
@overload
def aggregate(
        df:pd.DataFrame,
        xaxis_name:str,
        yaxis_name:str,
        yaxis_aggregate_method:str,
        caxis_name:str = None,
        )->tuple[pd.Series, pd.Series, pd.Series]:
    ...

def aggregate(
        df:pd.DataFrame,
        xagg_axis_name:str,
        yaxis_name:str,
        yaxis_aggregate_method:str,
        caxis_name:str|None = None,
        ):
    if caxis_name is None:
        aggregate_axis = [xagg_axis_name]
    else:
        aggregate_axis = [xagg_axis_name, caxis_name]
    if yaxis_aggregate_method == "count":
        df["__件数__"] = 1
        df_agg = df.groupby(aggregate_axis)["__件数__"].sum().reset_index()
        yaxis = df_agg["__件数__"]
    elif yaxis_aggregate_method == "mean":
        df_agg = df.groupby(aggregate_axis)[yaxis_name].mean().reset_index()
        yaxis = df_agg[yaxis_name]
    elif yaxis_aggregate_method == "median":
        df_agg = df.groupby(aggregate_axis)[yaxis_name].median().reset_index()
        yaxis = df_agg[yaxis_name]
    elif yaxis_aggregate_method == "nunique":
        df_agg = df.groupby(aggregate_axis)[yaxis_name].nunique().reset_index()
        yaxis = df_agg[yaxis_name]
    if caxis_name is None:
        return df_agg[xagg_axis_name], yaxis
    else:
        return df_agg[xagg_axis_name], yaxis, df_agg[caxis_name]

In [129]:
from bokeh.models import BoxSelectTool, ResetTool
def plot_vbar(
        source:ColumnDataSource,
        xagg_axis_name:str,
        yaxis_name:str,
        axis:Figure,
        caxis_name:str = None,
        caxis_values:list[str] = None,
        ):
    if caxis_name is None:
        axis.vbar(x=xagg_axis_name, top=yaxis_name, width=0.95, source=source, color="grey", alpha=0.3)
    else:
        renderers = []
        sources = []
        for ivalue, caxis_value in enumerate(caxis_values):
            view = CDSView(source=source, filters=[GroupFilter(column_name=caxis_name, group=caxis_value)])
            renderers.append(axis.vbar(x=xagg_axis_name, top=yaxis_name, width=0.8, view=view, legend_label = caxis_value, source=source, color=Category10[10][ivalue], alpha=0.5))
        hover = HoverTool(
            tooltips=[(xagg_axis_name, f"@{xagg_axis_name}"), (yaxis_name, f"@{yaxis_name}")],
            )
        axis.add_tools(hover)
        box_select = BoxSelectTool(renderers=renderers)
        reset_tool = ResetTool()
        axis.add_tools(box_select, reset_tool)
        
        axis.legend.click_policy="hide"
        axis.legend.location = "top_left"
    return axis

In [130]:
# box_select = BoxSelectTool()
# reset_tool = ResetTool()

# axis1.add_tools([box_select, reset_tool])

In [131]:
# source_whole1.data[xaxis_name1]

In [144]:
axis1 = figure(plot_width=600, plot_height=400, tools = [])
axis2 = figure(plot_width=600, plot_height=400, tools = [])
table = DataTable(source=ColumnDataSource(df), columns=[TableColumn(field="text", title="text")], width=1200, height=300)
plot = column(row(axis1, axis2), table)

use_caxis1 = True
use_caxis2 = True
normalize1 = False
normalize2 = False
show_whole1 = True
show_whole2 = True

if show_whole1:
    xagg_axis_whole1, yaxis_whole1 = aggregate(df, xaxis_name1, yaxis_name1, metric_name1)
    source_whole1 = ColumnDataSource(
        data = {
            xaxis_name1 : xagg_axis_whole1,
            yaxis_name1 : yaxis_whole1,
        }
    )
    plot_vbar(source_whole1, xaxis_name1, yaxis_name1, axis=axis1)
if show_whole2:
    xagg_axis_whole2, yaxis_whole2 = aggregate(df, xaxis_name2, yaxis_name2, metric_name2)
    source_whole2 = ColumnDataSource(
        data = {
            xaxis_name2 : xagg_axis_whole2,
            yaxis_name2 : yaxis_whole2,
        }
    )
    plot_vbar(source_whole2, xaxis_name2, yaxis_name2, axis=axis2)

if use_caxis1:
    caxis_values1 = sorted(df[caxis_name1].unique().tolist())
    xagg_axis1, yaxis1, caxis1 = aggregate(df, xaxis_name1, yaxis_name1, metric_name1, caxis_name1)
    source1 = ColumnDataSource(
        data = {
            xaxis_name1 : xagg_axis1,
            yaxis_name1 : yaxis1,
            caxis_name1 : caxis1,
        }
    )
    plot_vbar(source1, xaxis_name1, yaxis_name1, axis=axis1, caxis_name=caxis_name1, caxis_values=caxis_values1)
if use_caxis2:
    caxis_values2 = sorted(df[caxis_name2].unique().tolist())
    xagg_axis2, yaxis2, caxis2 = aggregate(df, xaxis_name2, yaxis_name2, metric_name2, caxis_name2)
    source2 = ColumnDataSource(
        data = {
            xaxis_name2 : xagg_axis2,
            yaxis_name2 : yaxis2,
            caxis_name2 : caxis2,
        }
    )
    plot_vbar(source2, xaxis_name2, yaxis_name2, axis=axis2, caxis_name=caxis_name2, caxis_values=caxis_values2)

js = CustomJS(
    args = {
        "source1": source1,
        "xaxis1": xagg_axis1,
        "caxis1": caxis1,
        },
    code = """
        console.log(source1.data);
        console.log(source1.selected.indices);
        source1.selected.indices.forEach(function(index){
            console.log(caxis1[index], xaxis1[index]);
        });
        """
)
source1.selected.js_on_change("indices", js)
axis1.js_on_event("reset", js)
save(plot, "test.html")

c:\Users\moja\AppData\Local\Programs\Python\Python310\lib\site-packages\bokeh\io\saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
c:\Users\moja\AppData\Local\Programs\Python\Python310\lib\site-packages\bokeh\io\saving.py:154: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'c:\\Users\\moja\\develop\\repos\\streamlit-bokeh\\test.html'